# Sprawozdanie z laboratorium 2

***Autor: Adam Dąbkowski***

## 0. Importowanie niezbędnych bibliotek

In [690]:
import copy

import numpy as np
import random
from utils import *

## 1. Implementacja algorytmu genetycznego

In [691]:
def genetic_algorithm(q, t_max, pc, pm, u):
    t = 0
    P = create_population()
    O = score(q, P)
    x_best, o_best = find_best(P, O)
    while t < t_max:
        S = selection(P)
        M = crossing_and_mutation(S, pc, pm, u)
        O = score(q, M)
        x, o = find_best(M, O)
        if o < o_best:
            x_best = x
            o_best = o
        P = M
        t+=1

In [692]:
P = create_population(100, 10)
P

array([[0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 1,

In [693]:
O = score(q, P)
O

[-4,
 -6,
 -1,
 -3,
 -4,
 -4,
 -7,
 -4,
 -4,
 -6,
 -4,
 -6,
 -5,
 -2,
 -4,
 -4,
 -6,
 -5,
 -5,
 -6,
 -7,
 -6,
 -3,
 -7,
 -6,
 -5,
 -6,
 -3,
 -6,
 -6,
 -5,
 -5,
 -3,
 -8,
 -3,
 -9,
 -4,
 -3,
 -3,
 -6,
 -3,
 -4,
 -3,
 -4,
 -4,
 -4,
 -2,
 -4,
 -4,
 -8,
 -6,
 -4,
 -4,
 -5,
 -4,
 -4,
 -6,
 -4,
 -2,
 -4,
 -4,
 -6,
 -6,
 -5,
 -1,
 -5,
 -3,
 -7,
 -3,
 -3,
 -4,
 -7,
 -5,
 -4,
 -5,
 -6,
 -4,
 -6,
 -5,
 -7,
 -5,
 -4,
 -5,
 -5,
 -3,
 -4,
 -8,
 -4,
 -6,
 -5,
 -5,
 -2,
 -3,
 -5,
 -4,
 -4,
 -6,
 -6,
 -4,
 -4]

In [694]:
find_best(P, O)

(array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), -1)

In [695]:
S = selection(P, O, 6)
S

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1]])

In [696]:
def crossing_and_mutation(S, pc, pm, u):
    C = []

    for i in range(0, u-1, 2):
        individual_1 = S[i]
        individual_2 = S[i+1]

        if random.random() < pc:
            crossing_point = random.randint(1, len(individual_1))
            individual_1_crossed = copy.deepcopy(individual_1)
            individual_2_crossed = copy.deepcopy(individual_2)

            print(f"Crossed: point = {crossing_point}")

            individual_1_crossed[crossing_point:] = individual_2[crossing_point:]
            individual_2_crossed[crossing_point:] = individual_1[crossing_point:]

            C.append(individual_1_crossed)
            C.append(individual_2_crossed)

            print(f"Crossed: point = {crossing_point}")
            print(f"Crossed: ind_1 = {individual_1}, {individual_1_crossed}")
            print(f"Crossed: ind_2 = {individual_2}, {individual_2_crossed}")

        else:
            C.append(individual_1)
            C.append(individual_2)

    if u % 2 == 1:
        C.append(S[-1])

    return np.array(C)

In [697]:
crossing_and_mutation(S, 0.7, 0.4, 5)

Crossed: point = 1
ind_1 = [0 1 1 1 1 1 0 1 0]
ind_2 = [1 1 0 1 1 0 0 0 0]
Crossed: ind_1 = [0 1 1 0 1 1 0 0 0 0], [0 1 1 0 1 1 0 0 0 0]
Crossed: ind_2 = [0 0 1 1 1 1 1 0 1 0], [0 0 1 1 1 1 1 0 1 0]
Crossed: point = 1
Crossed: ind_1 = [0 1 1 0 1 1 0 0 0 0], [0 0 1 1 1 1 1 0 1 0]
Crossed: ind_2 = [0 0 1 1 1 1 1 0 1 0], [0 1 1 0 1 1 0 0 0 0]


array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1]])